In [1]:
import tensorflow_datasets as tfds
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow import keras
from pathlib import Path
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil
import os

Using TensorFlow backend.


In [2]:
tf.compat.v1.estimator.WarmStartSettings

tensorflow_estimator.python.estimator.estimator.WarmStartSettings

In [3]:
tf.__version__

'1.15.0'

In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
(train_x, train_y), (test_x, test_y) = fashion_mnist.load_data()

In [6]:
train_x.shape

(60000, 28, 28)

In [7]:
n_classes = len(np.unique(train_y))

In [8]:
np.unique(train_y)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [9]:
train_y = train_y.reshape(-1, 1)

In [10]:
tf.__version__

'1.15.0'

In [11]:
notebook_path = os.path.abspath("iris.ipynb")

In [12]:
project_dir = Path(notebook_path).resolve().parents[1]
raw_data_dir = "{}/{}".format(project_dir, "data/raw")
processed_data_dir = "{}/{}".format(project_dir, "data/processed")
model_dir = "{}/{}".format(project_dir, "models")

In [13]:
def preprocess_image(image):

    image["x"] = tf.cast(image["x"], tf.float32)
    image["x"] /= 255
    
    return image

In [14]:
def reshape_and_cast_inputs(image, label):
    
    image = preprocess_image(image)
    label = tf.cast(label, tf.int32)

    return image, label

In [15]:
feature_columns = [tf.feature_column.numeric_column("x", shape=[28, 28])]

In [16]:
def train_input_fn(features, labels, batch_size, epochs, shuffle_size=1000):
    
    dataset = tf.data.Dataset.from_tensor_slices(({"x": features}, labels))
    dataset = dataset.map(reshape_and_cast_inputs)
    dataset = dataset.shuffle(shuffle_size).repeat(epochs).batch(batch_size)

    return dataset

In [17]:
def eval_input_fn(features, labels, batch_size, shuffle_size=1000):
    
    if labels is None:
        inputs = {"x": features}
    else:
        inputs = ({"x": features}, labels)

    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    
    if labels is None:
        dataset = dataset.map(preprocess_image)
    else:
        dataset = dataset.map(reshape_and_cast_inputs)
    
    dataset = dataset.cache().shuffle(shuffle_size).batch(batch_size)

    return dataset

In [18]:
def get_model():

    data_format="channels_last"
    input_shape = [28, 28, 1]
    
    l = tf.keras.layers

    max_pool = l.MaxPooling2D((2, 2), 
                              (2, 2), 
                              padding='same', 
                              data_format=data_format)

    return tf.keras.Sequential(
      [
          l.Conv2D(
              32,
              5,
              padding='same',
              data_format=data_format,
              activation=tf.nn.relu),
          max_pool,
          l.Conv2D(
              64,
              5,
              padding='same',
              data_format=data_format,
              activation=tf.nn.relu),
          max_pool,
          l.Flatten(),
          l.Dense(1024, activation=tf.nn.relu),
          l.Dropout(0.4),
          l.Dense(10)
      ])

In [19]:
def model_fn(features, labels, mode, params):
    
    print(features)
    features = tf.reshape(features["x"], [-1, 28, 28, 1])
    model = get_model()
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        logits = model(features, training=False)
    elif mode == tf.estimator.ModeKeys.EVAL:
        logits = model(features, training=False)
    elif mode == tf.estimator.ModeKeys.TRAIN:
        logits = model(features, training=True)
    predicted_classes = tf.argmax(logits, 1)

    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode=mode, 
                                          predictions=predictions)

    # Compute loss.
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, 
                                                  logits=logits)

    # Compute evaluation metrics.
    accuracy = tf.metrics.accuracy(labels=labels,
                                   predictions=predicted_classes,
                                   name='acc_op')
    
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])
    tf.summary.scalar('loss', loss)

    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode=mode, 
                                          loss=loss, 
                                          eval_metric_ops=metrics)

    # Create training op.
    assert mode == tf.estimator.ModeKeys.TRAIN
        
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

In [20]:
MODEL_NAME = "v1"
MODEL_DIR = "../model/{}".format(MODEL_NAME)

checkpoint_name = "model.ckpt-188"
checkpoint_path = "{}/{}".format(MODEL_DIR, checkpoint_name)
checkpoint_path = None

BATCH_SIZE = 32
EPOCHS = 10
nb_samples = train_x.shape[0]
steps_per_epoch = nb_samples // BATCH_SIZE
nb_evaluation_per_epoch = 10
TRAIN_STEPS = steps_per_epoch * EPOCHS
EVAL_STEPS = min(50, test_x.shape[0] // BATCH_SIZE)

In [21]:
print("Training samples: {}".format(nb_samples))
print("BATCH_SIZE: {}".format(BATCH_SIZE))
print("EPOCHS: {}".format(EPOCHS))
print("Steps per epoch: {}".format(steps_per_epoch))
print("TRAIN_STEPS: {}".format(TRAIN_STEPS))

Training samples: 60000
BATCH_SIZE: 32
EPOCHS: 10
Steps per epoch: 1875
TRAIN_STEPS: 18750


In [22]:
clear = True
if clear is True:
    shutil.rmtree(MODEL_DIR)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

In [23]:
training_config = tf.estimator.RunConfig(model_dir=MODEL_DIR, 
                                         save_summary_steps=10,
                                         keep_checkpoint_max=5,
                                         save_checkpoints_steps=steps_per_epoch / 10, 
                                         log_step_count_steps=steps_per_epoch / 10)

In [24]:
estimator = tf.estimator.Estimator(model_fn=model_fn, 
                                   config=training_config,
                                   warm_start_from=checkpoint_path)

INFO:tensorflow:Using config: {'_model_dir': '../model/v1', '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_steps': 187.5, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 187.5, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12e05beb8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [25]:
train_spec = tf.estimator.TrainSpec(lambda:train_input_fn(train_x, train_y, BATCH_SIZE, EPOCHS), 
                                    max_steps=TRAIN_STEPS)

eval_spec = tf.estimator.EvalSpec(lambda:eval_input_fn(test_x, test_y, BATCH_SIZE), 
                                  steps=EVAL_STEPS,
                                  throttle_secs=0) # no eval throttling: evaluates after each checkpoint

tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 187.5 or save_checkpoints_secs None.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
{'x': <tf.Tensor 'IteratorGetNext:0' shape=(?, 28, 28) dtype=float32>}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Savi

INFO:tensorflow:Calling model_fn.
{'x': <tf.Tensor 'IteratorGetNext:0' shape=(?, 28, 28) dtype=float32>}
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-18T20:33:31Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model/v1/model.ckpt-1128
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [5/50]
INFO:tensorflow:Evaluation [10/50]
INFO:tensorflow:Evaluation [15/50]
INFO:tensorflow:Evaluation [20/50]
INFO:tensorflow:Evaluation [25/50]
INFO:tensorflow:Evaluation [30/50]
INFO:tensorflow:Evaluation [35/50]
INFO:tensorflow:Evaluation [40/50]
INFO:tensorflow:Evaluation [45/50]
INFO:tensorflow:Evaluation [50/50]
INFO:tensorflow:Finished evaluation at 2020-05-18-20:33:33
INFO:tensorflow:Saving dict for global step 1128: accuracy = 0.82, global_step = 1128, loss = 0.48312408
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1128: ../model/v1/model.ckpt

INFO:tensorflow:Saving checkpoints for 2444 into ../model/v1/model.ckpt.
INFO:tensorflow:Calling model_fn.
{'x': <tf.Tensor 'IteratorGetNext:0' shape=(?, 28, 28) dtype=float32>}
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-18T20:35:57Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model/v1/model.ckpt-2444
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [5/50]
INFO:tensorflow:Evaluation [10/50]
INFO:tensorflow:Evaluation [15/50]
INFO:tensorflow:Evaluation [20/50]
INFO:tensorflow:Evaluation [25/50]
INFO:tensorflow:Evaluation [30/50]
INFO:tensorflow:Evaluation [35/50]
INFO:tensorflow:Evaluation [40/50]
INFO:tensorflow:Evaluation [45/50]
INFO:tensorflow:Evaluation [50/50]
INFO:tensorflow:Finished evaluation at 2020-05-18-20:35:58
INFO:tensorflow:Saving dict for global step 2444: accuracy = 0.839375, global_step = 2444, loss = 0.48019752
INFO:tensorflow:

INFO:tensorflow:loss = 0.2946608, step = 3573 (21.853 sec)
INFO:tensorflow:Saving checkpoints for 3760 into ../model/v1/model.ckpt.
INFO:tensorflow:Calling model_fn.
{'x': <tf.Tensor 'IteratorGetNext:0' shape=(?, 28, 28) dtype=float32>}
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-18T20:38:22Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model/v1/model.ckpt-3760
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [5/50]
INFO:tensorflow:Evaluation [10/50]
INFO:tensorflow:Evaluation [15/50]
INFO:tensorflow:Evaluation [20/50]
INFO:tensorflow:Evaluation [25/50]
INFO:tensorflow:Evaluation [30/50]
INFO:tensorflow:Evaluation [35/50]
INFO:tensorflow:Evaluation [40/50]
INFO:tensorflow:Evaluation [45/50]
INFO:tensorflow:Evaluation [50/50]
INFO:tensorflow:Finished evaluation at 2020-05-18-20:38:24
INFO:tensorflow:Saving dict for global step 3760: accuracy = 0.827

INFO:tensorflow:global_step/sec: 8.21165
INFO:tensorflow:loss = 0.4856978, step = 4889 (22.894 sec)
INFO:tensorflow:Saving checkpoints for 5076 into ../model/v1/model.ckpt.
INFO:tensorflow:Calling model_fn.
{'x': <tf.Tensor 'IteratorGetNext:0' shape=(?, 28, 28) dtype=float32>}
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-18T20:40:57Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model/v1/model.ckpt-5076
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [5/50]
INFO:tensorflow:Evaluation [10/50]
INFO:tensorflow:Evaluation [15/50]
INFO:tensorflow:Evaluation [20/50]
INFO:tensorflow:Evaluation [25/50]
INFO:tensorflow:Evaluation [30/50]
INFO:tensorflow:Evaluation [35/50]
INFO:tensorflow:Evaluation [40/50]
INFO:tensorflow:Evaluation [45/50]
INFO:tensorflow:Evaluation [50/50]
INFO:tensorflow:Finished evaluation at 2020-05-18-20:40:59
INFO:tensorflow:Saving di

...

INFO:tensorflow:Evaluation [50/50]
INFO:tensorflow:Finished evaluation at 2020-05-18-21:00:03
INFO:tensorflow:Saving dict for global step 14100: accuracy = 0.85, global_step = 14100, loss = 0.4472339
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 14100: ../model/v1/model.ckpt-14100
INFO:tensorflow:global_step/sec: 8.48428
INFO:tensorflow:loss = 0.3594095, step = 14101 (22.159 sec)
INFO:tensorflow:Saving checkpoints for 14288 into ../model/v1/model.ckpt.
INFO:tensorflow:Calling model_fn.
{'x': <tf.Tensor 'IteratorGetNext:0' shape=(?, 28, 28) dtype=float32>}
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-18T21:00:23Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model/v1/model.ckpt-14288
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [5/50]
INFO:tensorflow:Evaluation [10/50]
INFO:tensorflow:Evaluation [15/50]
INFO:tensorflow:Evaluatio

INFO:tensorflow:Evaluation [35/50]
INFO:tensorflow:Evaluation [40/50]
INFO:tensorflow:Evaluation [45/50]
INFO:tensorflow:Evaluation [50/50]
INFO:tensorflow:Finished evaluation at 2020-05-18-21:03:00
INFO:tensorflow:Saving dict for global step 15416: accuracy = 0.865, global_step = 15416, loss = 0.37713405
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 15416: ../model/v1/model.ckpt-15416
INFO:tensorflow:global_step/sec: 5.83297
INFO:tensorflow:loss = 0.37158975, step = 15417 (32.231 sec)
INFO:tensorflow:Saving checkpoints for 15604 into ../model/v1/model.ckpt.
INFO:tensorflow:Calling model_fn.
{'x': <tf.Tensor 'IteratorGetNext:0' shape=(?, 28, 28) dtype=float32>}
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-18T21:03:29Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model/v1/model.ckpt-15604
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evalu

INFO:tensorflow:Evaluation [20/50]
INFO:tensorflow:Evaluation [25/50]
INFO:tensorflow:Evaluation [30/50]
INFO:tensorflow:Evaluation [35/50]
INFO:tensorflow:Evaluation [40/50]
INFO:tensorflow:Evaluation [45/50]
INFO:tensorflow:Evaluation [50/50]
INFO:tensorflow:Finished evaluation at 2020-05-18-21:06:20
INFO:tensorflow:Saving dict for global step 16732: accuracy = 0.864375, global_step = 16732, loss = 0.409181
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 16732: ../model/v1/model.ckpt-16732
INFO:tensorflow:global_step/sec: 8.39349
INFO:tensorflow:loss = 0.2629088, step = 16733 (22.398 sec)
INFO:tensorflow:Saving checkpoints for 16920 into ../model/v1/model.ckpt.
INFO:tensorflow:Calling model_fn.
{'x': <tf.Tensor 'IteratorGetNext:0' shape=(?, 28, 28) dtype=float32>}
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-18T21:06:40Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model/v1/model.ckpt-16920

INFO:tensorflow:Evaluation [5/50]
INFO:tensorflow:Evaluation [10/50]
INFO:tensorflow:Evaluation [15/50]
INFO:tensorflow:Evaluation [20/50]
INFO:tensorflow:Evaluation [25/50]
INFO:tensorflow:Evaluation [30/50]
INFO:tensorflow:Evaluation [35/50]
INFO:tensorflow:Evaluation [40/50]
INFO:tensorflow:Evaluation [45/50]
INFO:tensorflow:Evaluation [50/50]
INFO:tensorflow:Finished evaluation at 2020-05-18-21:09:00
INFO:tensorflow:Saving dict for global step 18048: accuracy = 0.846875, global_step = 18048, loss = 0.4873652
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 18048: ../model/v1/model.ckpt-18048
INFO:tensorflow:global_step/sec: 8.5327
INFO:tensorflow:loss = 0.3136803, step = 18049 (22.032 sec)
INFO:tensorflow:Saving checkpoints for 18236 into ../model/v1/model.ckpt.
INFO:tensorflow:Calling model_fn.
{'x': <tf.Tensor 'IteratorGetNext:0' shape=(?, 28, 28) dtype=float32>}
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-18T21:09:33Z
INF

({'accuracy': 0.86125, 'loss': 0.5010213, 'global_step': 18750}, [])

In [26]:
def serving_input_receiver_fn():
    """
    input placeholder
    """
    inputs = {"x": tf.placeholder(shape=[28, 28], dtype=tf.float32)}
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)

In [27]:
export_dir = estimator.export_savedmodel(export_dir_base=MODEL_DIR, 
                                         serving_input_receiver_fn=serving_input_receiver_fn)

Instructions for updating:
This function has been renamed, use `export_saved_model` instead.
INFO:tensorflow:Calling model_fn.
{'x': <tf.Tensor 'Placeholder:0' shape=(28, 28) dtype=float32>}
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from ../model/v1/model.ckpt-18750
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ../model/v1/temp-b'1589807447'/saved_model.pb
